In [78]:
from components.feature_engineering import FeatureEngineering

In [79]:
feature_engineer = FeatureEngineering()

In [ ]:
def get_data(ticker, mode, target):
    result = feature_engineer.run(ticker=ticker, mode=mode)
    data = result[0]['output_1']['data']

    if mode == '1d':
        col_to_drop = ['date', 'middle_open_close', 'middle_high_low', 'open', 'high', 'low', 'close', 'volumn']
    else:
        col_to_drop = ['date_start', 'date_end', 'middle_open_close', 'middle_high_low', 'open', 'high', 'low', 'close', 'volumn']

    feature_columns = [col for col in data.columns if col not in col_to_drop]

    if mode == '1d':
        data = data[data['date'].dt.year >= 2023]
    else:
        data = data[data['date_start'].dt.year >= 2023]    


    X_test = data[feature_columns]
    y_test = data[target]

    return X_test, y_test



In [81]:
import json
with open("output/result.json", "r", encoding="utf-8") as file:
     json_data = json.load(file)

In [ ]:
import joblib
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
import numpy as np


for ticker_key, ticker_val in json_data.items():
    for mode_key, mode_val in ticker_val.items():
        for target_key, target_val in mode_val.items():
            X_test, y_test = get_data(ticker=ticker_key, mode=mode_key, target=target_key)
            model_path = f'output/{ticker_key}/{mode_key}/{target_key}/model.pkl'
            model = joblib.load(model_path)
            y_pred = model.predict(X_test)

            percentage_error =  (np.abs((y_test - y_pred) / y_test)) * 100 
            percent_changes = np.nan_to_num((y_pred[1:] - y_pred[:-1]) / y_pred[:-1] ) * 100 
            # percent_changes[0] = 0 

            target_val["percentage_error"] = percentage_error.tolist() 
            target_val["percent_changes"] = percent_changes.tolist() 
            

c:\Users\Admin\Desktop\DF\components\feature_engineering.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data = data.drop(columns=['ticker', 'type'])
c:\Users\Admin\Desktop\DF\components\feature_engineering.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data = data.drop(columns=['ticker', 'type'])
c:\Users\Admin\Desktop\DF\components\feature_engineering.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

In [ ]:
with open("output/result.json", "w", encoding="utf-8") as f:
    json.dump(json_data, f, ensure_ascii=False, indent=4)